In [2]:
from torchvision.models import vgg
from torch_lib.Model_heading_bin import *
from torch_lib.ClassAverages import *
from torchvision import transforms
import os, glob, cv2
from library.ron_utils import *
import matplotlib.pyplot as plt
import numpy as np

def plot_regressed_3d_bbox(img, cam_to_img, box_2d, dimensions, alpha, theta_ray, detectionid):

    # the math! returns X, the corners used for constraint
    location, X = calc_location(dimensions, cam_to_img, box_2d, alpha, theta_ray)

    orient = alpha + theta_ray

    #plot_2d_box(img, box_2d, detectionid)
    plot_3d_box(img, cam_to_img, orient, dimensions, location) # 3d boxes

    return location, orient

#REG PNG TO VIDEO
import cv2, os, glob
def make_video(folder, img_paths, reg_folder, fps=15):
    reg_folder = os.path.join(folder, reg_folder)
    video_name = os.path.join(folder, f'{reg_folder}.avi')
    image0 = os.path.join(folder, img_paths[0])
    frame = cv2.imread(image0)
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name, 0, fps=fps, frameSize=(width,height))
    for i in range(len(img_paths)):
        path = os.path.join(folder, img_paths[i].replace('images', reg_folder))
        video.write(cv2.imread(path))
    cv2.destroyAllWindows()
    video.release()
    
def make_compare_video(folder, img_paths, reg_folder1, reg_folder2, fps=15):
    video_name = os.path.join(folder, f'C_{reg_folder1}_{reg_folder2}.avi')
    image0 = os.path.join(folder, img_paths[0])
    frame = cv2.imread(image0)
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name, 0, fps=fps, frameSize=(width*2, height))
    for i in range(len(img_paths)):
        path1 = os.path.join(folder, img_paths[i].replace('images', reg_folder1))
        path2 = os.path.join(folder, img_paths[i].replace('images', reg_folder2))
        img1 = cv2.imread(path1)
        img2 = cv2.imread(path2)
        compare = np.concatenate((img1, img2), axis=1) #horizontal
        video.write(compare)
    cv2.destroyAllWindows()
    video.release()

def round_list(list_, float_point=2):
    foo = list()
    for i in range(len(list_)):
        new_val = round(float(list_[i]), float_point)
        foo.append(new_val)
    return foo

# Eval all folders

In [10]:
import json
# init setting
weights_list = ['DA_B4_N1_50', 'all_BL_B4_N1_50']
ELAN_averages = ClassAverages(average_file='all_ELAN_class_averages.txt')
cam_to_img = np.array([
        [1.418667e+03, 0.000e+00, 6.4e+02, 0],
        [0.000e+00, 1.418867e+03, 3.6e+02, 0],
        [0.000e+00, 000e+00, 1.0e+00, 0] ])
class_dict = {8:'truck', 9:'car', 10:'motor', 11:'bike'}
device = torch.device('cuda') # 選gpu的index

for weights_name in weights_list:
    weights_path = f'weights/{weights_name}.pkl'

    checkpoint = torch.load(weights_path, map_location=device) #if training on 2 GPU, mapping on the same device
    normalize_type = checkpoint['normal']
    bin_num = checkpoint['bin']
    angle_per_class = 2*np.pi/float(bin_num)

    my_vgg = vgg.vgg19_bn(weights='DEFAULT').to(device)
    model = Model(features=my_vgg.features, bins=bin_num).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    # for img processing
    if normalize_type == 0:
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    if normalize_type == 1:
        normalize = transforms.Normalize(mean=[0.596, 0.612, 0.587], std=[0.256, 0.254, 0.257])
    process = transforms.Compose([transforms.ToTensor(), 
                                  transforms.Resize([224,224], transforms.InterpolationMode.BICUBIC), 
                                  normalize])


    valid_folder = 'IVAvalid_for_NCTU'
    for time in ['day', 'night']:
        folder = f'{valid_folder}/{time}'
        for sub_f in os.listdir(folder):
            sub_folder = os.path.join(folder, sub_f)
            with open(f'{sub_folder}/TEST_images.json') as f:
                img_paths = json.load(f)
            label_json = f'{sub_folder}/TEST_objects.json'
            with open(label_json) as f:
                label_dicts = json.load(f)

            os.makedirs(os.path.join(sub_folder, weights_name), exist_ok=True)
            result = list()
            model.eval()
            for i in range(len(img_paths)):

                path = os.path.join(sub_folder, img_paths[i])
                img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
                label_dict = label_dicts[i]

                save_path = path.replace('images', weights_name)
                result_json = os.path.join(sub_folder, f'REG_result_{weights_name}.json')
                result_dict = dict()
                result_dict['alpha'] = list()
                result_dict['theta_ray'] = list()
                result_dict['dimension'] = list()
                result_dict['location'] = list()
                for j in range(len(label_dict['labels'])):
                    difficulty = label_dict['difficulties'][j] # 1 is hard to distinguish
                    if difficulty == 0:
                        color = (255, 0, 0)
                        line_width = 2
                    else:
                        color = (0, 0, 255)
                        line_width = 1
                    class_ = class_dict[label_dict['labels'][j]] # 5:行人,6:機車騎士,7:腳踏車騎士,8:大車,9:小車,10:機車,11:腳踏車
                    object_id = label_dict['id'][j] # for tracking
                    box_2d = label_dict['boxes'][j]
                    box_2d = [[box_2d[0], box_2d[1]], [box_2d[2],box_2d[3]]]
                    left_top = box_2d[0]
                    right_btm = box_2d[1]

                    crop = img[left_top[1]:right_btm[1]+1, left_top[0]:right_btm[0]+1] 
                    crop = process(crop) 
                    crop = torch.stack([crop]).to(device)

                    [RESIDUALs, BIN_CONFs, delta_DIMs] = model(crop)
                    bin_argmax = torch.max(BIN_CONFs, dim=1)[1]
                    orient_residual = RESIDUALs[torch.arange(len(RESIDUALs)), bin_argmax] 
                    Alphas = angle_per_class*bin_argmax + orient_residual #mapping bin_class and residual to get alpha
                    reg_alpha = float(Alphas[0].data)
                    reg_alpha = round(angle_correction(reg_alpha), 2)
                    reg_dim = delta_DIMs.cpu().data.numpy()[0, :]
                    reg_dim += ELAN_averages.get_item(class_)
                    
                    theta_ray = round(calc_theta_ray(img.shape[1], box_2d, cam_to_img), 2)
                    loc, ry = plot_regressed_3d_bbox(img, cam_to_img, box_2d, reg_dim, reg_alpha, theta_ray, object_id)
                    reg_dim = round_list(reg_dim, 2)
                    loc = round_list(loc, 2)
                    result_dict['alpha'].append(reg_alpha)
                    result_dict['theta_ray'].append(theta_ray)
                    result_dict['dimension'].append(reg_dim)
                    result_dict['location'].append(loc)
                    
                result.append(result_dict)
                #plt.imsave(save_path, img)
                #plt.show()
            with open(result_json, 'w') as f:
                json.dump(result, f)
            print(f'DONE {sub_folder}')
            #make_video(sub_folder, img_paths, weights_name, fps=15)
print('ALL FINISHED')

DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-091818-824
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092149-828
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092421-831
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143139-2863
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143946-2887
DONE IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-202628-892
DONE IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-202954-900
DONE IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203217-905
DONE IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203246-908
DONE IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203322-911
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-091818-824
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092149-828
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092421-831
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143139-2863
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143946-2887
DONE IVAvalid_for_NCTU/night\c32b95371b6b

### make COMPARE video only (given folder)

In [6]:

import cv2, os, glob
valid_folder = 'IVAvalid_for_NCTU'
for time in ['day', 'night']:
    folder = f'{valid_folder}/{time}'
    for sub_f in os.listdir(folder):
        sub_folder = os.path.join(folder, sub_f)
        with open(f'{sub_folder}/TEST_images.json') as f:
            img_paths = json.load(f)
        make_compare_video(sub_folder, img_paths, weights_list[0], weights_list[1])
        print(f'Done {sub_folder}')
print('ALL FINISHED')

Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-091818-824
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092149-828
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092421-831
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143139-2863
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143946-2887
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-202628-892
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-202954-900
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203217-905
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203246-908
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203322-911
ALL FINISHED


## Reproduce by REG_result.json

In [18]:
import json
# init setting
weights_list = ['DA_B4_N1_50', 'all_BL_B4_N1_50']
cam_to_img = np.array([
        [1.418667e+03, 0.000e+00, 6.4e+02, 0],
        [0.000e+00, 1.418867e+03, 3.6e+02, 0],
        [0.000e+00, 000e+00, 1.0e+00, 0] ])
class_dict = {8:'truck', 9:'car', 10:'motor', 11:'bike'}
device = torch.device('cuda') # 選gpu的index

valid_folder = 'IVAvalid_for_NCTU'
for weights_name in weights_list:
    for time in ['day', 'night']:
        folder = f'{valid_folder}/{time}'
        for sub_f in os.listdir(folder):
            sub_folder = os.path.join(folder, sub_f)
            with open(f'{sub_folder}/TEST_images.json') as f:
                img_paths = json.load(f)
            label_json = f'{sub_folder}/TEST_objects.json'
            with open(label_json) as f:
                label_dicts = json.load(f)
            
            result_json = f'{sub_folder}/REG_result_{weights_name}.json'
            with open(result_json) as f:
                result_dicts = json.load(f)
            
            re_folder = os.path.join(sub_folder, f'R_{weights_name}')
            os.makedirs(re_folder, exist_ok=True)
            for i in range(len(img_paths)):
                path = os.path.join(sub_folder, img_paths[i])
                save_path = path.replace('images', f'R_{weights_name}')
                
                img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
                label_dict = label_dicts[i]
                result_values = result_dicts[i]
                for j in range(len(label_dict['labels'])):
                    orient = result_values['alpha'][j] + result_values['theta_ray'][j]
                    dim = result_values['dimension'][j]
                    loc = result_values['location'][j]
                    plot_3d_box(img, cam_to_img, orient, dim, loc) # 3d boxes
                    #plt.imshow(img)
                    #plt.show()
                plt.imsave(save_path, img)
            make_video(sub_folder, img_paths, f'R_{weights_name}', fps=15)
            print(f'Done {weights_name} | {sub_folder.split('\\')[-1]')
print('All finished')

Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-091818-824
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092149-828
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092421-831
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143139-2863
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143946-2887
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-202628-892
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-202954-900
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203217-905
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203246-908
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203322-911
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-091818-824
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092149-828
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092421-831
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143139-2863
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143946-2887
Done IVAvalid_for_NCTU/night\c32b95371b6b